# Chains

## 1_chains_basics

In [13]:
%pip install -q python-dotenv langchain langchain-community langchain-ollama

Note: you may need to restart the kernel to use updated packages.


In [14]:
from dotenv import load_dotenv

In [15]:
# Load environment variables from .env
load_dotenv()

True

In [16]:
from langchain_ollama import ChatOllama

In [17]:
model = ChatOllama(
        base_url="http://localhost:11434",
        model="llama3.2",
        temperature=0)

In [24]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

In [45]:
# Define prompt templates (no need for separate Runnable chains)
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a comedian who tells jokes about {topic}."),
        ("human", "Tell me {joke_count} jokes."),
    ]
)

In [51]:
# Create the combined chain using LangChain Expression Language (LCEL)
chain = prompt_template | model

In [ ]:
# Create the combined chain using .pipe() method
chain = prompt_template.pipe(model)

In [52]:
# Run the chain
result = chain.invoke({"topic": "lawyers", "joke_count": 3})

In [53]:
# Output
print(result)

content='Here are three lawyer-themed jokes for you:\n\n1. Why did the lawyer\'s client bring a ladder to the courtroom?\n\nBecause he wanted to take his case to a higher court! (get it?)\n\n2. Why did the lawyer\'s wife leave him?\n\nBecause she couldn\'t take his constant "objection" to their relationship!\n\n3. What did the lawyer say to the coffee machine in the office break room?\n\n"You\'re always brewing up trouble, but I\'m going to have to sue you for emotional distress!"' additional_kwargs={} response_metadata={'model': 'llama3.2', 'created_at': '2024-11-10T21:31:24.060522506Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 10342761537, 'load_duration': 27722983, 'prompt_eval_count': 41, 'prompt_eval_duration': 97000000, 'eval_count': 104, 'eval_duration': 10216000000} id='run-26e78236-fc05-4350-97e4-1cbf71cce7c8-0' usage_metadata={'input_tokens': 41, 'output_tokens': 104, 'total_tokens': 145}


In [15]:
# Create the combined chain using LangChain Expression Language (LCEL)
chain = prompt_template | model | StrOutputParser()

In [16]:
result = chain.invoke({"topic": "lawyers", "joke_count": 3})

In [49]:
# Output
print(result)

content='Here are three lawyer-themed jokes for you:\n\n1. Why did the lawyer\'s client bring a ladder to the courtroom?\n\nBecause he wanted to take his case to a higher court! (get it?)\n\n2. Why did the lawyer\'s wife leave him?\n\nBecause she couldn\'t take his constant "objection" to their relationship!\n\n3. What did the lawyer say when his client asked him how much it would cost to sue someone?\n\n"Don\'t worry, we\'ll bill you by the hour... and then some!"' additional_kwargs={} response_metadata={'model': 'llama3.2', 'created_at': '2024-11-10T21:29:53.986792135Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 11411358597, 'load_duration': 20991038, 'prompt_eval_count': 41, 'prompt_eval_duration': 997000000, 'eval_count': 104, 'eval_duration': 10391000000} id='run-60ea568d-e7f3-4f21-b59f-fa0fe4ccc684-0' usage_metadata={'input_tokens': 41, 'output_tokens': 104, 'total_tokens': 145}


## 2_chains_under_the_hood

In [18]:
from langchain.schema.runnable import RunnableLambda, RunnableSequence

In [19]:
# Create individual runnables (steps in the chain)
format_prompt = RunnableLambda(lambda x: prompt_template.format_prompt(**x))
invoke_model = RunnableLambda(lambda x: model.invoke(x.to_messages()))
parse_output = RunnableLambda(lambda x: x.content)

In [20]:
# Create the RunnableSequence (equivalent to the LCEL chain)
chain = RunnableSequence(first=format_prompt, middle=[invoke_model], last=parse_output)

In [21]:
# Run the chain
response = chain.invoke({"topic": "lawyers", "joke_count": 3})

In [22]:
# Output
print(response)

Here are three lawyer-themed jokes for you:

1. Why did the lawyer's client bring a ladder to the courtroom?

Because he wanted to take his case to a higher court! (get it?)

2. Why did the lawyer's wife leave him?

Because she couldn't take his constant "objection" to their relationship!

3. What did the lawyer say to the coffee machine in the office break room?

"You're always brewing up trouble, but I'm going to have to sue you for emotional distress!"


## 3_chains_extended

In [23]:
# Define additional processing steps using RunnableLambda
uppercase_output = RunnableLambda(lambda x: x.upper())
count_words = RunnableLambda(lambda x: f"Word count: {len(x.split())}\n{x}")

In [24]:
# Create the combined chain using LangChain Expression Language (LCEL)
chain = prompt_template | model | StrOutputParser() | uppercase_output | count_words

In [25]:
# Run the chain
result = chain.invoke({"topic": "lawyers", "joke_count": 3})

In [26]:
# Output
print(result)

Word count: 81
HERE ARE THREE LAWYER-THEMED JOKES FOR YOU:

1. WHY DID THE LAWYER'S CLIENT BRING A LADDER TO THE COURTROOM?

BECAUSE HE WANTED TO TAKE HIS CASE TO A HIGHER COURT! (GET IT?)

2. WHY DID THE LAWYER'S WIFE LEAVE HIM?

BECAUSE SHE COULDN'T TAKE HIS CONSTANT "OBJECTION" TO THEIR RELATIONSHIP!

3. WHAT DID THE LAWYER SAY TO THE COFFEE MACHINE IN THE OFFICE BREAK ROOM?

"YOU'RE ALWAYS BREWING UP TROUBLE, BUT I'M GOING TO HAVE TO SUE YOU FOR EMOTIONAL DISTRESS!"


## 4_chains_parallel

In [18]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableParallel, RunnableLambda

In [19]:
# Define prompt template
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert product reviewer."),
        ("human", "List the main features of the product {product_name}."),
    ]
)

In [20]:
# Define pros analysis step
def analyze_pros(features):
    pros_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert product reviewer."),
            (
                "human",
                "Given these features: {features}, list the pros of these features.",
            ),
        ]
    )
    return pros_template.format_prompt(features=features)

In [21]:
# Define cons analysis step
def analyze_cons(features):
    cons_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert product reviewer."),
            (
                "human",
                "Given these features: {features}, list the cons of these features.",
            ),
        ]
    )
    return cons_template.format_prompt(features=features)

In [22]:
# Combine pros and cons into a final review
def combine_pros_cons(pros, cons):
    return f"Pros:\n{pros}\n\nCons:\n{cons}"

In [25]:
# Simplify branches with LCEL
pros_branch_chain = (
    RunnableLambda(lambda x: analyze_pros(x)) | model | StrOutputParser()
)

In [26]:
cons_branch_chain = (
    RunnableLambda(lambda x: analyze_cons(x)) | model | StrOutputParser()
)

In [27]:
# Create the combined chain using LangChain Expression Language (LCEL)
chain = (
    prompt_template
    | model
    | StrOutputParser()
    | RunnableParallel(branches={"pros": pros_branch_chain, "cons": cons_branch_chain})
    | RunnableLambda(lambda x: combine_pros_cons(x["branches"]["pros"], x["branches"]["cons"]))
)

In [28]:
# Run the chain
result = chain.invoke({"product_name": "MacBook Pro"})

In [29]:
# Output
print(result)

Pros:
Based on the provided features, here are some pros of the MacBook Pro:

1. **High-Resolution Display**: The Retina display with high-resolution (up to 5K) and vibrant colors provides an excellent visual experience for watching videos, browsing the web, and working on creative projects.

2. **Durable Aluminum Unibody Construction**: The aluminum unibody construction is both durable and lightweight, making it easy to carry around without sacrificing performance or protection.

3. **Long Battery Life**: With up to 20 hours of web browsing on a single charge, the MacBook Pro offers an impressive battery life that can last throughout a full day, even with heavy usage.

4. **Fast Charging Capabilities**: The fast charging capabilities allow for quick top-ups, making it convenient to keep your laptop charged and ready to go whenever you need it.

5. **High-Performance Computing**: With up to 10th or 11th Gen Intel Core i9 processor, the MacBook Pro offers high-performance computing capa

## 5_chains_branching

In [35]:
from langchain.schema.runnable import RunnableBranch

In [30]:
# Define prompt templates for different feedback types
positive_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human",
         "Generate a thank you note for this positive feedback: {feedback}."),
    ]
)

In [31]:
negative_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human",
         "Generate a response addressing this negative feedback: {feedback}."),
    ]
)

In [32]:
neutral_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human",
            "Generate a request for more details for this neutral feedback: {feedback}.",
        ),
    ]
)

In [33]:
escalate_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human",
            "Generate a message to escalate this feedback to a human agent: {feedback}.",
        ),
    ]
)

In [34]:
# Define the feedback classification template
classification_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human",
         "Classify the sentiment of this feedback as positive, negative, neutral, or escalate: {feedback}."),
    ]
)

In [36]:
# Define the runnable branches for handling feedback
branches = RunnableBranch(
    (
        lambda x: "positive" in x,
        positive_feedback_template | model | StrOutputParser()  # Positive feedback chain
    ),
    (
        lambda x: "negative" in x,
        negative_feedback_template | model | StrOutputParser()  # Negative feedback chain
    ),
    (
        lambda x: "neutral" in x,
        neutral_feedback_template | model | StrOutputParser()  # Neutral feedback chain
    ),
    escalate_feedback_template | model | StrOutputParser()
)


In [37]:
# Create the classification chain
classification_chain = classification_template | model | StrOutputParser()

In [38]:
# Combine classification and response generation into one chain
chain = classification_chain | branches

In [ ]:
# Run the chain with an example review
# Good review - "The product is excellent. I really enjoyed using it and found it very helpful."
# Bad review - "The product is terrible. It broke after just one use and the quality is very poor."
# Neutral review - "The product is okay. It works as expected but nothing exceptional."
# Default - "I'm not sure about the product yet. Can you tell me more about its features and benefits?"

In [39]:
review = "The product is terrible. It broke after just one use and the quality is very poor."
result = chain.invoke({"feedback": review})

In [40]:
# Output the result
print(result)

Here is a sample message you can use to escalate the feedback to a human agent:

"Hi [Agent's Name], 

I'm reaching out regarding a recent interaction with your system. The user provided feedback that I've classified as 'Negative'. I'd like to discuss this further and explore ways to improve our response to similar situations.

Could we schedule a call to review the feedback and discuss potential next steps? I'd appreciate the opportunity to provide more context and get your input on how to address this issue.

Please let me know a convenient time for you, or if there's any additional information you need from me beforehand.

Thank you,
[Your Name]"

This message provides context, explains the reason for escalating the feedback, and requests a call with the agent to discuss further.
